In [1]:
import tensorflow as tf
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn; seaborn.set()

/home/x-val/anaconda3/envs/tensorflow/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## Data

In [2]:
# XOR

num_train = 64000
num_test = 10000

def make_dataset(X, y):
    train_X = np.array([])
    train_y = np.array([], dtype=int)

    for i in range(int(num_train/4)):
        train_X = np.append(train_X, X)
        train_y = np.append(train_y, y)

    train_X = train_X.reshape((num_train, 2))
    train_y = train_y.reshape((num_train, 2))

    test_indices = np.random.choice(num_train, num_test, replace=True)
    test_X = train_X[test_indices]
    test_y = train_y[test_indices]

    print(train_X.shape, train_y.shape)
    print(test_X.shape, test_y.shape)

    return train_X, train_y, test_X, test_y

#### XOR
- 0, 0 = 0
- 0, 1 = 1
- 1, 0 = 1
- 1, 1 = 0

In [3]:
xor_X = [[0, 0], [0, 1], [1, 0], [1, 1]]
xor_y = [[1, 0], [0, 1], [0, 1], [1, 0]] # One Hot

train_X, train_y, test_X, test_y = make_dataset(xor_X, xor_y)

print(train_X[:10])
print(train_y[:10])

(64000, 2) (64000, 2)
(10000, 2) (10000, 2)
[[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  1.]]
[[1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]]


In [4]:
# Parameters
learning_rate = 0.0001
num_epochs = 5
batch_size = 64

input_size = 2
num_classes = 2

num_h1 = 256
num_h2 = 512
num_h3 = 1024

## Model

In [5]:
X = tf.placeholder(tf.float32, [None, input_size])
y = tf.placeholder(tf.float32, [None, num_classes])

W1 = tf.get_variable('W1', [input_size, num_h1], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable('b1', [num_h1], initializer=tf.constant_initializer(0.0))

W2 = tf.get_variable('W2', [num_h1, num_h2], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable('b2', [num_h2], initializer=tf.constant_initializer(0.0))

W3 = tf.get_variable('W3', [num_h2, num_h3], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable('b3', [num_h3], initializer=tf.constant_initializer(0.0))

W4 = tf.get_variable('W4', [num_h3, num_classes], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.get_variable('b4', [num_classes], initializer=tf.constant_initializer(0.0))

In [6]:
# Layer
input_layer = tf.add(tf.matmul(X, W1), b1)
input_layer = tf.nn.relu(input_layer)

hidden_layer1 = tf.add(tf.matmul(input_layer, W2), b2)
hidden_layer1 = tf.nn.relu(hidden_layer1)

hidden_layer2 = tf.add(tf.matmul(hidden_layer1, W3), b3)
hidden_layer2 = tf.nn.relu(hidden_layer2)

output_layer = tf.add(tf.matmul(hidden_layer2, W4), b4)

In [7]:
# Construct model
pred = output_layer

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## Train

In [8]:
num_steps = num_train // batch_size
traininig_losses = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        traininig_loss = 0
        for steps in range(num_steps):
            feed_dict = {X: train_X[steps*batch_size:(steps+1)*batch_size], y: train_y[steps*batch_size:(steps+1)*batch_size]}
            _, l = sess.run([optimizer, cost], feed_dict=feed_dict)
            traininig_loss += l
            
            if steps != 0 and steps % 200 == 0:
                print("Average trainig loss for Epoch", epoch+1, " (" + str(steps) + "):", traininig_loss/steps)

    # Test
    test_accuarcy = 0;
    for i in range(0, num_test, 50):
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        test_accuarcy += accuracy.eval({X: test_X[i:i+50], y: test_y[i:i+50]})
    print("Accuracy:", test_accuarcy/(num_test/50))

Average trainig loss for Epoch 1  (200): 0.214880046877
Average trainig loss for Epoch 1  (400): 0.108745440566
Average trainig loss for Epoch 1  (600): 0.072642528154
Average trainig loss for Epoch 1  (800): 0.0545207863064
Average trainig loss for Epoch 2  (200): 3.80547518307e-05
Average trainig loss for Epoch 2  (400): 2.92624546455e-05
Average trainig loss for Epoch 2  (600): 2.3581103654e-05
Average trainig loss for Epoch 2  (800): 1.96799277347e-05
Average trainig loss for Epoch 3  (200): 4.12686565824e-06
Average trainig loss for Epoch 3  (400): 3.62179314948e-06
Average trainig loss for Epoch 3  (600): 3.22495199763e-06
Average trainig loss for Epoch 3  (800): 2.9037461934e-06
Average trainig loss for Epoch 4  (200): 1.28596914863e-06
Average trainig loss for Epoch 4  (400): 1.17614775206e-06
Average trainig loss for Epoch 4  (600): 1.08395936905e-06
Average trainig loss for Epoch 4  (800): 1.00236320158e-06
Average trainig loss for Epoch 5  (200): 5.66541966975e-07
Average tr